In [1]:
import tensorflow as tf
from tensorflow import keras

import numpy as np

print(tf.__version__)

2.15.0


#1. Load Data

In [2]:
imdb = keras.datasets.imdb
(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000)

17464789/17464789 [==============================] - 0s 0us/step


In [3]:
len(train_data), len(test_data)

(25000, 25000)

In [4]:
len(train_data[0]), len(train_data[102])

(218, 790)

In [5]:
word_index = imdb.get_word_index()

1641221/1641221 [==============================] - 0s 0us/step


In [7]:
word_index = {k:(v+3) for k,v in word_index.items()}
word_index["<PAD>"] = 0
word_index["<START>"] = 1
word_index["<UNK>"] = 2  # unknown
word_index["<UNUSED>"] = 3

In [9]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

In [10]:
decode_review(train_data[0])

"<START> this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert <UNK> is an amazing actor and now the same being director <UNK> father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for <UNK> and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also <UNK> to the two little boy's that played the <UNK> of norman and paul they were just brilliant children are often left out of the <UNK> list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and should be praised for wh

#2. Text Classification

In [11]:
train_data = keras.preprocessing.sequence.pad_sequences(train_data, value=word_index['<PAD>'], padding='post', maxlen=256)
test_data = keras.preprocessing.sequence.pad_sequences(test_data, value=word_index['<PAD>'], padding='post', maxlen=256)

In [12]:
len(train_data[0]), len(train_data[102])

(256, 256)

In [13]:
train_data[0]

array([   1,   14,   22,   16,   43,  530,  973, 1622, 1385,   65,  458,
       4468,   66, 3941,    4,  173,   36,  256,    5,   25,  100,   43,
        838,  112,   50,  670,    2,    9,   35,  480,  284,    5,  150,
          4,  172,  112,  167,    2,  336,  385,   39,    4,  172, 4536,
       1111,   17,  546,   38,   13,  447,    4,  192,   50,   16,    6,
        147, 2025,   19,   14,   22,    4, 1920, 4613,  469,    4,   22,
         71,   87,   12,   16,   43,  530,   38,   76,   15,   13, 1247,
          4,   22,   17,  515,   17,   12,   16,  626,   18,    2,    5,
         62,  386,   12,    8,  316,    8,  106,    5,    4, 2223, 5244,
         16,  480,   66, 3785,   33,    4,  130,   12,   16,   38,  619,
          5,   25,  124,   51,   36,  135,   48,   25, 1415,   33,    6,
         22,   12,  215,   28,   77,   52,    5,   14,  407,   16,   82,
          2,    8,    4,  107,  117, 5952,   15,  256,    4,    2,    7,
       3766,    5,  723,   36,   71,   43,  530,  4

In [14]:
vocab_size = 10000

model = keras.Sequential()
model.add(keras.layers.Embedding(vocab_size, 16, input_shape=(None,)))
model.add(keras.layers.GlobalAveragePooling1D())
model.add(keras.layers.Dense(16, activation='relu'))
model.add(keras.layers.Dense(1, activation='sigmoid'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 16)          160000    
                                                                 
 global_average_pooling1d (  (None, 16)                0         
 GlobalAveragePooling1D)                                         
                                                                 
 dense (Dense)               (None, 16)                272       
                                                                 
 dense_1 (Dense)             (None, 1)                 17        
                                                                 
Total params: 160289 (626.13 KB)
Trainable params: 160289 (626.13 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [15]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [16]:
particial_x_train = train_data[10000:]
x_val = train_data[:10000]

particial_y_train = train_labels[10000:]
y_val = train_labels[:10000]

In [17]:
history = model.fit(particial_x_train, particial_y_train, epochs=50, batch_size=512,
                    validation_data=(x_val, y_val))

Epoch 1/50
30/30 [==============================] - 3s 55ms/step - loss: 0.6922 - accuracy: 0.5501 - val_loss: 0.6906 - val_accuracy: 0.6301
Epoch 2/50
30/30 [==============================] - 1s 38ms/step - loss: 0.6874 - accuracy: 0.6453 - val_loss: 0.6841 - val_accuracy: 0.7407
Epoch 3/50
30/30 [==============================] - 1s 37ms/step - loss: 0.6770 - accuracy: 0.7657 - val_loss: 0.6710 - val_accuracy: 0.7501
Epoch 4/50
30/30 [==============================] - 1s 31ms/step - loss: 0.6581 - accuracy: 0.7653 - val_loss: 0.6487 - val_accuracy: 0.7647
Epoch 5/50
30/30 [==============================] - 1s 30ms/step - loss: 0.6286 - accuracy: 0.7853 - val_loss: 0.6172 - val_accuracy: 0.7777
Epoch 6/50
30/30 [==============================] - 1s 24ms/step - loss: 0.5885 - accuracy: 0.8021 - val_loss: 0.5763 - val_accuracy: 0.7948
Epoch 7/50
30/30 [==============================] - 1s 23ms/step - loss: 0.5421 - accuracy: 0.8183 - val_loss: 0.5331 - val_accuracy: 0.8011
Epoch 8/50
30

In [18]:
results = model.evaluate(test_data, test_labels)
print(results)

782/782 [==============================] - 3s 3ms/step - loss: 0.3682 - accuracy: 0.8674
[0.3681679666042328, 0.8674399852752686]
